# GWAS Preparation

In [1]:
pwd()

'/home/hnasrulloh/Works/pangenomes/notebooks'

In [2]:
from os import path
from collections import defaultdict

import pandas as pd

In [3]:
DATA_DIR = path.join(path.pardir, 'data')

GWAS_DIR = path.join(DATA_DIR, 'gwas')

## Generating _pheno_ tsv files

In [4]:
AMR_EXCEL = path.join(DATA_DIR, 'amr_ready.xlsx')
df = pd.read_excel(AMR_EXCEL, dtype=str)

df = df.loc[:, ['genome_id', 'antibiotic', 'phenotype']]

pheno = { 'R': 1, 'S': 0 }
df['phenotype'] = df['phenotype'].map(pheno)

# Match the samples name within gene_presence_absence.Rtab
samples = df.genome_id.map(lambda name: name + '.PATRIC')
df.insert(0, 'samples', samples)
df = df.drop(['genome_id'], axis=1)

for antibiotic in ['ethambutol', 'rifampin', 'isoniazid']:
    df_ab = df[df.antibiotic == antibiotic]
    df_ab = df_ab.drop(['antibiotic'], axis=1)
    df_ab.columns = ['samples', antibiotic]

    ANTIBIOTIC_TSV = path.join(DATA_DIR, f'pheno_{antibiotic}.tsv')
    df_ab.to_csv(ANTIBIOTIC_TSV, index=False, sep='\t')